In [1]:
from invoke_llm import process_prompt

prompt = """
create a new folder /my_first
"""
await process_prompt(prompt)



INFO     invoke_llm - 🤖 Processing prompt: 
create a new folder /my_first
...
DEBUG    invoke_llm - 🌐 Calling Anthropic API...
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: message_start
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: content_block_start
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: input_json
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: input_json
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: input_json
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: input_json
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: content_block_stop
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: message_delta
DEBUG    invoke_llm - ⚠️ Unhandled chunk type: message_stop
DEBUG    invoke_llm - 📝 Complete response: 
INFO     invoke_llm - ✅ Successfully processed prompt


In [73]:

from anthropic import Anthropic
import os
from dotenv import load_dotenv
import prompts

load_dotenv()

anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

def text_response():  # renamed from text_stream since it's no longer streaming
    message = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1024,
        temperature=0.7,
        messages=[{
            "role": "user",
            "content": "write a python script that prints hello world"
        }],
        system=prompts.RESPONSE_DETECTOR_PROMPT,
        stream=False  # changed from True to False
    )
    
    return message.content  # directly return the complete message content

text_response()

[TextBlock(text='[clipboard]', type='text')]

In [69]:
import os
from groq import Groq
from dotenv import load_dotenv
import prompts

load_dotenv()

client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),  # This is the default and can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": prompts.RESPONSE_DETECTOR_PROMPT
        },
        {
            "role": "user",
            "content": "write a python script that prints hello world"
        }
    ],
    model="llama3-70b-8192",
)
print(chat_completion.choices[0].message.content)

copy
